In [ ]:
import gensim.downloader
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

In [ ]:
print(list(gensim.downloader.info()['models'].keys()))

In [ ]:
data = pd.read_csv('synonyms.csv', sep=",",header=0)
data

In [ ]:
models = ['word2vec-google-news-300', 
          'glove-wiki-gigaword-300', 'fasttext-wiki-news-subwords-300',
          'glove-twitter-200', 'glove-twitter-100']
performances = {model:[] for model in models}
for model in models:
    word2vec = gensim.downloader.load(model)
    L_pred = []
    for row in data.iterrows():
        if row[1]["question"] in word2vec.key_to_index and row[1]["answer"] in word2vec.key_to_index:
            prediction = []
            if row[1]["0"] in word2vec.key_to_index:
                prediction.append([row[1]['0'], word2vec.similarity(row[1]['0'], row[1]['question'])]) 

            if row[1]["1"] in word2vec.key_to_index:
                prediction.append([row[1]['1'], word2vec.similarity(row[1]['1'], row[1]['question'])])

            if row[1]["2"] in word2vec.key_to_index:
                prediction.append([row[1]['2'], word2vec.similarity(row[1]['2'], row[1]['question'])])

            if row[1]["3"] in word2vec.key_to_index:
                prediction.append([row[1]['3'], word2vec.similarity(row[1]['3'], row[1]['question'])])

            if len(prediction)!=0:
                guess = max(prediction, key=lambda x: x[1])
                if guess[0] == row[1]["answer"]:
                    label='correct'
                else:
                    label='wrong'
            else:
                label = 'guess'
                guess = [row[1][str(np.random.randint(0,4))]]

        else:
            label = 'guess'
            guess = [row[1][str(np.random.randint(0,4))]]
        s = F"{row[1]['question']},{row[1]['answer']},{guess[0]},{label}"
        L_pred.append(s)
    performances[model] = L_pred

In [ ]:
for model in models:
    with open(f"{model}-details.csv",'w') as f:
        for pred in performances[model]:
            f.write(pred+'\n')

In [ ]:
performances = {model:0 for model in models}
for i, model in enumerate(models):
    L_pred = []
    with open(f"{model}-details.csv",'r') as f:
        for line in f.readlines():
            L_pred.append(line.split(','))
            L_pred[-1][-1] = L_pred[-1][-1].rstrip("\n")
    C = list(map(lambda x:x[-1],L_pred)).count("correct")
    V = list(map(lambda x:x[-1],L_pred)).count("wrong") + C
    performances[model] = C/V
    if i==0:
        with open("analysis.csv", "w") as f:
            f.write(f"{model},{len(word2vec.key_to_index)},{C},{V},{C/V}\n")
            print(f"{model},{len(word2vec.key_to_index)},{C},{V},{C/V}")
    else:
        with open("analysis.csv", "a") as f:
            f.write(f"{model},{len(word2vec.key_to_index)},{C},{V},{C/V}\n")
            print(f"{model},{len(word2vec.key_to_index)},{C},{V},{C/V}")

In [ ]:
random.seed(0)
L_pred = list()
for row in data.iterrows():
    random_pred = row[1][str(random.randint(1,3))]
    if random_pred == row[1]["answer"]:
        label='correct'
    else:
        label='wrong'
    s = F"{row[1]['question']},{row[1]['answer']},{random_pred},{label}"
    L_pred.append(s)

with open(f"random-baseline-details.csv",'w') as f:
    for pred in L_pred:
        f.write(pred+'\n')
        print(pred)

In [ ]:
with open(f"random-baseline-details.csv",'r') as f:
    for line in f.readlines():
        L_pred.append(line.split(','))
        L_pred[-1][-1] = L_pred[-1][-1].rstrip("\n")
C = list(map(lambda x:x[-1],L_pred)).count("correct")
V = list(map(lambda x:x[-1],L_pred)).count("wrong") + C
performances['random-baseline'] = C/V
performances['human-gold-standard'] = 0.8557
performances
